In [1]:
import pandas as pd
import numpy as np

In [2]:
airports = pd.read_csv("./airports.csv")
countries = pd.read_csv("./countries.csv", keep_default_na = False)
routes = pd.read_csv("./routes.csv", header=None)

In [3]:
airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [4]:
countries.head()

,id,code,name,continent,wikipedia_link,keywords
0,302672,AD,Andorra,EU,https://en.wikipedia.org/wiki/Andorra,
1,302618,AE,United Arab Emirates,AS,https://en.wikipedia.org/wiki/United_Arab_Emir...,"UAE,مطارات في الإمارات العربية المتحدة"
2,302619,AF,Afghanistan,AS,https://en.wikipedia.org/wiki/Afghanistan,
3,302722,AG,Antigua and Barbuda,NA,https://en.wikipedia.org/wiki/Antigua_and_Barbuda,
4,302723,AI,Anguilla,NA,https://en.wikipedia.org/wiki/Anguilla,


In [5]:
routes.columns=['Airline', 'AirlineID', 'Source Airport', 'Source Airport ID',
                'Destination Airport', 'Destination Airport ID', 'Codeshare', 'Stops', 'Equipment']
routes.head()

,Airline,AirlineID,Source Airport,Source Airport ID,Destination Airport,Destination Airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [6]:
routes = routes.drop(routes[routes['Stops']>0].index)

In [7]:
routes = routes.drop(columns=['Airline', 'AirlineID', 'Source Airport ID', 'Destination Airport ID',
                              'Codeshare', 'Stops', 'Equipment'])
airports_clean =  airports.drop(columns=['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
                                         'elevation_ft', 'continent', 'iso_region','municipality', 
                                         'scheduled_service', 'gps_code','local_code', 'home_link', 
                                         'wikipedia_link', 'keywords'])

In [8]:
routes = routes.merge(airports_clean, left_on='Source Airport', right_on='iata_code')
routes = routes.merge(airports_clean, left_on='Destination Airport', right_on='iata_code')

In [9]:
routes = routes.drop(columns=['Source Airport', 'Destination Airport', 'iata_code_x', 'iata_code_y' ])
routes = routes.drop(routes[routes['iso_country_x'].isna()].index)
routes = routes.drop(routes[routes['iso_country_y'].isna()].index)

In [10]:
routes_internacionales = routes.query('iso_country_x != iso_country_y').copy()

In [11]:
routes_internacionales = routes_internacionales.merge(countries[['code', 'continent']], 
                                                      left_on='iso_country_x', right_on='code')
routes_internacionales = routes_internacionales.merge(countries[['code', 'continent']], 
                                                      left_on='iso_country_y', right_on='code')

In [12]:
routes_internacionales = routes_internacionales.drop(routes_internacionales
                                                     [routes_internacionales['continent_x'].isna()].index)
routes_internacionales = routes_internacionales.drop(routes_internacionales
                                                     [routes_internacionales['continent_y'].isna()].index)

In [13]:
routes_internacionales = routes_internacionales.drop(columns=['iso_country_x', 'iso_country_y', 'code_x', 'code_y'])


In [14]:
routes_internacionales.columns=['Procedencia', 'Destino']

In [15]:
routes_internacionales['Procedencia'] = routes_internacionales['Procedencia'].map({'AS':'Asia',
                                                                                   'EU':'Europa',
                                                                                   'AF':'Africa',
                                                                                   'NA':'América Norte',
                                                                                   'SA':'América Sur',
                                                                                   'OC':'Oceanía'},
                                                                                  na_action=None)

routes_internacionales['Destino'] = routes_internacionales['Destino'].map({'AS':'Asia',
                                                                           'EU':'Europa',
                                                                           'AF':'Africa',
                                                                           'NA':'América Norte',
                                                                           'SA':'América Sur',
                                                                           'OC':'Oceanía'},
                                                                           na_action=None)

In [16]:
routes_internacionales = routes_internacionales.assign(Ruta=1)
data_final = routes_internacionales.groupby(['Procedencia', 'Destino']).count()

In [17]:
data_final.to_csv('chord_data.csv')